In [1]:
import re
import ucsc_restapi as upi
import numpy as np
import pathlib
cwd = pathlib.Path.cwd()
import pandas
import pickle
import random
import GeneClass as Gene
from GData import NormalData

In [2]:
min_ant: int = 100
min_pos: int = 100

sense_data_1 = cwd.parent / "Data_Files" / "Primates" / "Genetics" / "Homo_sapiens" / "Sense_Data" / "Data_Set_1_cleaned_dict.pkl"
sense_data_2 = cwd.parent / "Data_Files" / "Primates" / "Genetics" / "Homo_sapiens" / "Sense_Data" / "Data_Set_2_cleaned_dict.pkl"
sense_save = cwd.parent / "Data_Files" / "Primates" / "Genetics" / "Homo_sapiens" / "Sense_Data" / "Sense_cleaned_dict.pkl"

output_path = cwd.parent / "Data_Files" / "Primates" / "Genetics" / "Homo_sapiens" / "G_Data_v2"
output_path.mkdir(parents = True, exist_ok = True)

with open(sense_data_1, "rb") as file:
    sense_data_1 = pickle.load(file)

with open(sense_data_2, "rb") as file:
    sense_data_2 = pickle.load(file)

sense_data = sense_data_1 | sense_data_2

with open(sense_save, "wb") as file:
    pickle.dump(sense_data, file)

In [3]:
anti_data_1 = cwd.parent / "Data_Files" / "Primates" / "Genetics" / "Homo_sapiens" / "Anti_Data" / "Data_Set_1_cleaned_dict.pkl"
anti_data_2 = cwd.parent / "Data_Files" / "Primates" / "Genetics" / "Homo_sapiens" / "Anti_Data" / "Data_Set_2_cleaned_dict.pkl"
anti_save = cwd.parent / "Data_Files" / "Primates" / "Genetics" / "Homo_sapiens" / "Anti_Data" / "Anti_cleaned_dict.pkl"

with open(anti_data_1, "rb") as file:
    anti_data_1 = pickle.load(file)

with open(anti_data_2, "rb") as file:
    anti_data_2 = pickle.load(file)

anti_data = anti_data_1 | anti_data_2

with open(anti_save, "wb") as file:
    pickle.dump(anti_data, file)

For all data, sense and antisense

In [4]:
NormalData(cwd.parent / "Data_Files" / "Primates" / "Genetics" / "Homo_sapiens" / "Cleaned_Data" / f"Cleaned_Data_dict.pkl",
           output_path / f"Exon_all_{min_pos}-{min_ant}.xlsx",
           output_path / f"Intron_all_{min_pos}-{min_ant}.xlsx",
           min_ant_length = min_ant,
           min_pos_length = min_pos)

For sense data

In [5]:
NormalData(cwd.parent / "Data_Files" / "Primates" / "Genetics" / "Homo_sapiens" / "Sense_Data" / f"Sense_cleaned_dict.pkl",
           output_path / f"Exon_sense_{min_pos}-{min_ant}.xlsx",
           output_path / f"Intron_sense_{min_pos}-{min_ant}.xlsx",
           min_ant_length = min_ant,
           min_pos_length = min_pos)

For anti sense data

In [6]:
NormalData(cwd.parent / "Data_Files" / "Primates" / "Genetics" / "Homo_sapiens" / "Anti_Data" / f"Anti_cleaned_dict.pkl",
           output_path / f"Exon_antisense_{min_pos}-{min_ant}.xlsx",
           output_path / f"Intron_antisense_{min_pos}-{min_ant}.xlsx",
           min_ant_length = min_ant,
           min_pos_length = min_pos)

Because the antisense data is not reversed but that's what Dr G wants...

In [7]:
exon_anti_xlsx = output_path / f"Exon_antisense_{min_pos}-{min_ant}.xlsx"
exon_anti_xlsx_v2 = output_path / f"Exon_antisense_v2_{min_pos}-{min_ant}.xlsx"
intron_anti_xlsx = output_path / f"Intron_antisense_{min_pos}-{min_ant}.xlsx"
intron_anti_xlsx_v2 = output_path / f"Intron_antisense_v2_{min_pos}-{min_ant}.xlsx"

In [8]:
exon_anti = pandas.read_excel(exon_anti_xlsx, sheet_name = "sheet1")
intron_anti = pandas.read_excel(intron_anti_xlsx, sheet_name = "sheet1")

print(exon_anti.columns)
print(intron_anti.columns)

Index(['Name', 'Chrm', 'Strand', 'Posterior_100', 'Exon', 'Anterior_100'], dtype='object')
Index(['Name', 'Chrm', 'Strand', 'Posterior_100', 'Intron', 'Anterior_100'], dtype='object')


In [9]:
exon_anti[f"Posterior_{min_pos}"] = exon_anti[f"Posterior_{min_pos}"].apply(lambda x: x[::-1])
exon_anti[f"Anterior_{min_ant}"] = exon_anti[f"Anterior_{min_ant}"].apply(lambda x: x[::-1])
exon_anti["Exon"] = exon_anti["Exon"].apply(lambda x: x[::-1])

exon_anti.to_excel(exon_anti_xlsx_v2, sheet_name = "sheet1")

In [10]:
intron_anti[f"Posterior_{min_pos}"] = intron_anti[f"Posterior_{min_pos}"].apply(lambda x: x[::-1])
intron_anti[f"Anterior_{min_ant}"] = intron_anti[f"Anterior_{min_ant}"].apply(lambda x: x[::-1])
intron_anti["Intron"] = intron_anti["Intron"].apply(lambda x: x[::-1])

intron_anti.to_excel(intron_anti_xlsx_v2, sheet_name = "sheet1")

In [12]:
normal2normal = pandas.read_excel(output_path / f"Intron_all_{min_pos}-{min_ant}.xlsx", sheet_name = "sheet1")
del normal2normal["Intron"]
normal2normal["Posterior_Length"] = normal2normal[f"Posterior_{min_pos}"].apply(lambda x: len(x))
normal2normal["Anterior_Length"] = normal2normal[f"Anterior_{min_ant}"].apply(lambda x: len(x))

In [13]:
print(normal2normal.head())
print(normal2normal.shape)

             Name  Chrm Strand  \
0  NM_001375617.1  chr1      +   
1  NM_001375617.1  chr1      +   
2  NM_001375617.1  chr1      +   
3  NM_001375617.1  chr1      +   
4  NM_001375617.1  chr1      +   

                                       Posterior_100  \
0  TACAGGCAAGAACCAGGATGAATGTGTGATTGCTTTGCATGACTGC...   
1  TCAGGCTCCTTCTCTGGCCCAGCCTCTGGTGTTCAGTAATTCGAAG...   
2  CTGGCTCAGTTGGCAGCTCAGCATTCTCAGTCTGGAAGCACCACCA...   
3  ACAGATGTCGCCTGGATCTTCAGACAACCAGTCCTCTAGCCCTCAG...   
4  AGTTTGGGTCAGAGCCTGTCCTTTCTGATTATGAGTCCACCCCCAC...   

                                        Anterior_100  Posterior_Length  \
0  CATTCCTGGGAGATGGTCGGGAAGAAGAAGGGAGTCTCAGGCCAGA...               100   
1  GTGAGCATGTTAGGGAAAGGATTTGGTGATGTCGGTGAAGCTAAAG...               100   
2  ATTTGAAGAACCCAAGTGATTCAGCAGTGCACAGCCCCTTTACAAA...               100   
3  ATTCCTGCTCTGGCTGTGGAGATGCCTGGCTCAGCAGATATCTCAG...               100   
4  TGAATCATCCTCTACAATTTCATCTAACCAGAGTCAGGAGTCTGGT...               100   

   Anterior_L

In [14]:
normal2normal_filter = normal2normal[normal2normal["Anterior_Length"] == 100]
print(normal2normal.shape)

(18140, 7)


In [15]:
# print(normal2normal[["Strand", "Anterior_Length"]].head(n = 20))
print(normal2normal["Anterior_Length"].min())

100


In [16]:
normal2normal.to_excel(output_path / f"Normal_{min_pos}-{min_ant}.xlsx", sheet_name = "Sheet 1")